# Solution for Advent of Code 2025 – Part 2

The correct solution for this problem is based on integer linear programming, which requires the use of the `Z3` library.

Initially, I tried to solve the problem using `linprog` from `scipy`, which performs continuous linear programming over real numbers. This approach works for some inputs and test data, but for other inputs it produces an `X` vector where all elements are real numbers instead of integers. 

I also attempted to handle this by taking the floor and ceil of fractional values, generating all possible combinations, and checking against the target. However, this method did not reliably produce valid integer solutions.

Nevertheless, I have included this linprog approach in the notebook as a reference and for comparison.

In [ ]:
import re

from scipy.optimize import linprog
from z3 import Int, Optimize, Sum

# Read Input Data

Place your input data into a file called `input.txt` in the same folder as this Jupyter Notebook.

In [ ]:
buttons = []
joltage = []

with open("input.txt", "r") as f:
    for line in f:
        pattern = re.search(r"\[(.*?)\]", line).group(1)
        buttons_strings = re.findall(r"\((.*?)\)", line)
        buttons_set = [list(map(int, p.split(','))) for p in buttons_strings]

        joltage_string = re.search(r"\{(.*?)\}", line).group(1)
        joltage_target = list(map(int, joltage_string.split(',')))

        buttons.append(buttons_set)
        joltage.append(joltage_target)

# Integer Linear Programming (ILP) Solution based on Z3

This solution uses integer linear programming with the `Z3` library and will work for any input data.

In [ ]:
total_result = 0

for index_machine in range(len(buttons)):
    machine_buttons = buttons[index_machine]  # list of lists: each button → affected states
    machine_joltage = joltage[index_machine]  # target vector

    num_buttons = len(machine_buttons)
    num_states = len(machine_joltage)

    x = [Int(f"x_{index_machine}_{i}") for i in range(num_buttons)]

    optimizer = Optimize()

    for variable in x:
        optimizer.add(variable >= 0)

    for state_idx in range(num_states):
        button_contributions = []
        for j in range(num_buttons):
            if state_idx in machine_buttons[j]:
                button_contributions.append(x[j])

        constraint = Sum(button_contributions)
        optimizer.add(constraint == machine_joltage[state_idx])

    optimizer.minimize(Sum(x))

    optimizer.check()
    
    model = optimizer.model()
    presses = sum(model[variable].as_long() for variable in x)
    total_result += presses

print("Minimum button presses to configure all machines:", total_result)

# Continuous Linear Programming Solution based on SciPy (Might Not Work)

The solution works for some inputs and test data, but for other inputs it produces an `X` vector where the elements are real numbers instead of integers. Because the problem requires integer button presses, this method may not give valid solutions in all cases.

In [ ]:
total_result = 0
trustable_solution = True

for index_machine in range(len(buttons)):
    machine_buttons = buttons[index_machine]
    machine_joltage = joltage[index_machine]

    A = [ [0] * len(machine_buttons) for _ in range(len(machine_joltage)) ]

    for btn_index, btn_states in enumerate(machine_buttons):
        for s in btn_states:
            A[s][btn_index] = 1

    T = machine_joltage
    c = [1 for _ in range(len(machine_buttons))]

    bounds = [(0, None)] * len(machine_buttons)
    result = linprog(c, A_eq=A, b_eq=T, bounds=bounds, method='highs')
    total_result += result.fun

    if any(abs(v - round(v)) > 1e-6 for v in result.x):
        trustable_solution = False
        print("WARNING: Solution contains non-integer values: ", result.x.tolist(), result.fun)

if trustable_solution is True:
    print("[Correct Answer] Minimum button to press to configure joltage:", total_result)

else:
    print("[Answer Might Be Wrong] Minimum button to press to configure joltage:", total_result)


In [ ]:
total_result = 0

for index_machine in range(len(buttons)):
    machine_buttons = buttons[index_machine]  # list of lists: each button → affected states
    machine_joltage = joltage[index_machine]  # target vector

    num_buttons = len(machine_buttons)
    num_states = len(machine_joltage)

    x = [Int(f"x_{index_machine}_{i}") for i in range(num_buttons)]

    optimizer = Optimize()

    for variable in x:
        optimizer.add(variable >= 0)

    for state_idx in range(num_states):
        button_contributions = []
        for j in range(num_buttons):
            if state_idx in machine_buttons[j]:
                button_contributions.append(x[j])

        constraint = Sum(button_contributions)
        optimizer.add(constraint == machine_joltage[state_idx])

    optimizer.minimize(Sum(x))

    optimizer.check()
    
    model = optimizer.model()
    presses = sum(model[variable].as_long() for variable in x)
    total_result += presses

print("Minimum button presses to configure all machines:", total_result)